# 🔍 Diagnóstico de Conectividad Frontend-Backend

Este notebook verificará si el backend está funcionando correctamente y el frontend puede conectarse.

In [1]:
# 1. Verificar dependencias básicas del sistema
import os
import sys
import subprocess
import requests
from pathlib import Path

print("🔍 VERIFICACIÓN DEL SISTEMA")
print("=" * 50)

# Verificar estructura de proyecto
current_dir = Path.cwd()
print(f"📁 Directorio actual: {current_dir}")

# Verificar archivos críticos
critical_files = [
    "backend/Embedding.py",
    "backend/api/main.py", 
    "frontend/src/app/page.tsx",
    "frontend/package.json"
]

print("\n📋 Archivos críticos:")
for file in critical_files:
    file_path = current_dir.parent / file if current_dir.name == "backend" else current_dir / file
    exists = "✅" if file_path.exists() else "❌"
    print(f"   {exists} {file}")

# Verificar variables de entorno
print("\n🔑 Variables de entorno:")
api_key = os.getenv("OPENAI_API_KEY")
print(f"   OPENAI_API_KEY: {'✅ Configurada' if api_key else '❌ No configurada'}")

print("\n🐍 Python y dependencias:")
print(f"   Python: {sys.version}")

# Verificar si podemos importar módulos críticos
try:
    import fastapi
    print("   ✅ FastAPI instalado")
except ImportError:
    print("   ❌ FastAPI no instalado")

try:
    import langchain
    print("   ✅ LangChain instalado")
except ImportError:
    print("   ❌ LangChain no instalado")

try:
    from langchain_openai import OpenAIEmbeddings
    print("   ✅ OpenAI embeddings disponibles")
except ImportError:
    print("   ❌ OpenAI embeddings no disponibles")

🔍 VERIFICACIÓN DEL SISTEMA
📁 Directorio actual: c:\Users\asus\OneDrive - Universidad San Francisco de Quito\Documentos\GitHub\tendering_app\backend

📋 Archivos críticos:
   ✅ backend/Embedding.py
   ✅ backend/api/main.py
   ✅ frontend/src/app/page.tsx
   ✅ frontend/package.json

🔑 Variables de entorno:
   OPENAI_API_KEY: ❌ No configurada

🐍 Python y dependencias:
   Python: 3.13.5 (tags/v3.13.5:6cb20a2, Jun 11 2025, 16:15:46) [MSC v.1943 64 bit (AMD64)]
   ❌ FastAPI no instalado
   ✅ LangChain instalado
   ✅ OpenAI embeddings disponibles


In [3]:
# 2. Verificar si el API está corriendo
print("\n🌐 VERIFICACIÓN DEL API BACKEND")
print("=" * 50)

api_url = "http://localhost:8000"
endpoints_to_test = [
    "/",
    "/docs", 
    "/health",
    "/api/v1/health"
]

api_running = False

for endpoint in endpoints_to_test:
    try:
        response = requests.get(f"{api_url}{endpoint}", timeout=5)
        if response.status_code == 200:
            print(f"   ✅ {endpoint} - OK ({response.status_code})")
            api_running = True
        else:
            print(f"   ⚠️  {endpoint} - {response.status_code}")
    except requests.exceptions.ConnectionError:
        print(f"   ❌ {endpoint} - No se puede conectar")
    except requests.exceptions.Timeout:
        print(f"   ⚠️  {endpoint} - Timeout")
    except Exception as e:
        print(f"   ❌ {endpoint} - Error: {e}")

if not api_running:
    print("\n❌ EL API NO ESTÁ CORRIENDO")
    print("🚀 Para iniciar el API, ejecuta en una terminal:")
    print("   cd backend")
    print("   fastapi dev api/main.py --host 0.0.0.0 --port 8000")
else:
    print("\n✅ API está corriendo correctamente")

# Verificar procesos corriendo
print(f"\n🔍 Verificando procesos de FastAPI...")
try:
    # En Windows usar tasklist
    result = subprocess.run(['tasklist', '/FI', 'IMAGENAME eq python.exe'], 
                          capture_output=True, text=True, shell=True)
    if 'python.exe' in result.stdout:
        print("   ✅ Proceso Python ejecutándose")
        # Buscar líneas que contengan FastAPI o uvicorn
        lines = result.stdout.split('\n')
        python_processes = [line for line in lines if 'python.exe' in line]
        print(f"   📊 Procesos Python activos: {len(python_processes)}")
    else:
        print("   ❌ No se encontraron procesos Python")
except Exception as e:
    print(f"   ⚠️  Error verificando procesos: {e}")


🌐 VERIFICACIÓN DEL API BACKEND
   ✅ / - OK (200)
   ✅ /docs - OK (200)
   ✅ /health - OK (200)
   ✅ /api/v1/health - OK (200)

✅ API está corriendo correctamente

🔍 Verificando procesos de FastAPI...
   ✅ Proceso Python ejecutándose
   📊 Procesos Python activos: 4


In [7]:
# 3. Verificar el frontend
print("\n🎨 VERIFICACIÓN DEL FRONTEND")
print("=" * 50)

frontend_dir = current_dir.parent / "frontend" if current_dir.name == "backend" else current_dir / "frontend"
print(f"📁 Directorio frontend: {frontend_dir}")

if frontend_dir.exists():
    print("   ✅ Directorio frontend existe")
    
    # Verificar archivos críticos del frontend
    frontend_files = [
        "package.json",
        "src/app/page.tsx",
        "src/app/dashboard/page.tsx"
    ]
    
    for file in frontend_files:
        file_path = frontend_dir / file
        exists = "✅" if file_path.exists() else "❌"
        print(f"   {exists} {file}")
    
    # Verificar si el frontend está corriendo
    frontend_urls = [
        "http://localhost:3000",  # Next.js dev server
        "http://localhost:8888"   # Python simple server
    ]
    
    frontend_running = False
    for url in frontend_urls:
        try:
            response = requests.get(url, timeout=3)
            if response.status_code == 200:
                print(f"   ✅ Frontend corriendo en {url}")
                frontend_running = True
                break
        except:
            continue
    
    if not frontend_running:
        print("   ❌ Frontend no está corriendo")
        print("   🚀 Para iniciar el frontend:")
        print("      Opción 1 (Next.js): cd frontend && npm run dev")
        print("      Opción 2 (Simple): python -m http.server 8888 --directory frontend")
    
    # Leer package.json para ver dependencias
    package_json = frontend_dir / "package.json"
    if package_json.exists():
        try:
            import json
            with open(package_json, 'r', encoding='utf-8') as f:
                package_data = json.load(f)
            
            print(f"\n📦 Información del proyecto frontend:")
            print(f"   Nombre: {package_data.get('name', 'N/A')}")
            print(f"   Versión: {package_data.get('version', 'N/A')}")
            
            scripts = package_data.get('scripts', {})
            if scripts:
                print("   📜 Scripts disponibles:")
                for script_name in scripts:
                    print(f"      - {script_name}")
                    
        except Exception as e:
            print(f"   ⚠️ Error leyendo package.json: {e}")
            
else:
    print("   ❌ Directorio frontend no existe")


🎨 VERIFICACIÓN DEL FRONTEND
📁 Directorio frontend: c:\Users\asus\OneDrive - Universidad San Francisco de Quito\Documentos\GitHub\tendering_app\frontend
   ✅ Directorio frontend existe
   ✅ package.json
   ✅ src/app/page.tsx
   ✅ src/app/dashboard/page.tsx
   ✅ Frontend corriendo en http://localhost:8888

📦 Información del proyecto frontend:
   Nombre: dragonchat
   Versión: 0.1.0
   📜 Scripts disponibles:
      - dev
      - build
      - start
      - lint


In [5]:
# 4. Probar conectividad específica frontend-backend
print("\n🔗 PRUEBA DE CONECTIVIDAD FRONTEND-BACKEND")
print("=" * 50)

# Simular las llamadas que haría el frontend
test_endpoints = [
    {"url": "http://localhost:8000/api/v1/health", "description": "Health check"},
    {"url": "http://localhost:8000/api/v1/documents/list", "description": "Lista de documentos"},
    {"url": "http://localhost:8000/api/v1/analysis/status", "description": "Estado del análisis"}
]

for test in test_endpoints:
    try:
        response = requests.get(test["url"], timeout=10)
        if response.status_code == 200:
            print(f"   ✅ {test['description']}: OK")
            try:
                data = response.json()
                if isinstance(data, dict) and len(data) > 0:
                    print(f"      📄 Respuesta: {list(data.keys())}")
            except:
                print(f"      📄 Respuesta: {response.text[:100]}...")
        else:
            print(f"   ❌ {test['description']}: HTTP {response.status_code}")
            print(f"      📄 Error: {response.text[:100]}")
    except requests.exceptions.ConnectionError:
        print(f"   ❌ {test['description']}: No se puede conectar al backend")
    except Exception as e:
        print(f"   ❌ {test['description']}: {e}")

print("\n💡 SOLUCIONES PARA PROBLEMAS COMUNES")
print("=" * 50)

print("Si el backend no está corriendo:")
print("   1. Abre una terminal en el directorio del proyecto")
print("   2. cd backend")
print("   3. fastapi dev api/main.py --host 0.0.0.0 --port 8000")

print("\nSi el frontend no puede conectarse:")
print("   1. Verifica que el backend esté en puerto 8000")
print("   2. Revisa las URLs en el código del frontend")
print("   3. Verifica que no haya problemas de CORS")

print("\nSi hay errores de dependencias:")
print("   1. pip install -r requirements.txt")
print("   2. Configura las variables de entorno (.env)")
print("   3. npm install (en el directorio frontend)")

print("\nPara debuggear más:")
print("   1. Revisa los logs del backend en la terminal")
print("   2. Abre las herramientas de desarrollador en el navegador")
print("   3. Verifica la pestaña Network para ver las llamadas HTTP")


🔗 PRUEBA DE CONECTIVIDAD FRONTEND-BACKEND
   ✅ Health check: OK
      📄 Respuesta: ['status', 'version', 'timestamp', 'analysis_available', 'cache_size', 'directories_ok']
   ✅ Lista de documentos: OK
      📄 Respuesta: ['status', 'total_documents', 'documents']
   ✅ Estado del análisis: OK
      📄 Respuesta: ['status', 'analysis_available', 'dependencies_ok', 'active_analyses', 'cached_systems', 'timestamp', 'message']

💡 SOLUCIONES PARA PROBLEMAS COMUNES
Si el backend no está corriendo:
   1. Abre una terminal en el directorio del proyecto
   2. cd backend
   3. fastapi dev api/main.py --host 0.0.0.0 --port 8000

Si el frontend no puede conectarse:
   1. Verifica que el backend esté en puerto 8000
   2. Revisa las URLs en el código del frontend
   3. Verifica que no haya problemas de CORS

Si hay errores de dependencias:
   1. pip install -r requirements.txt
   2. Configura las variables de entorno (.env)
   3. npm install (en el directorio frontend)

Para debuggear más:
   1. Revis

# RESUMEN FINAL - SISTEMA OPERATIVO

Tu sistema de análisis de licitaciones está ahora completamente funcional!

In [8]:
# 5. Resumen final y información de acceso
import requests
from datetime import datetime

print("ESTADO FINAL DEL SISTEMA TENDERING AI")
print("=" * 60)

# Estado de servicios
services = [
    {
        "name": "API Backend",
        "url": "http://localhost:8000",
        "description": "Análisis de documentos, IA, embeddings"
    },
    {
        "name": "Frontend Web",
        "url": "http://localhost:8888",
        "description": "Interfaz de usuario, dashboard"
    },
    {
        "name": "API Documentation",
        "url": "http://localhost:8000/docs",
        "description": "Documentación interactiva Swagger"
    }
]

print("\nSERVICIOS DISPONIBLES:")
print("-" * 40)
for service in services:
    try:
        response = requests.get(service["url"], timeout=3)
        status = "ACTIVO" if response.status_code == 200 else f"ERROR {response.status_code}"
        print(f"✅ {service['name']}: {status}")
        print(f"   URL: {service['url']}")
        print(f"   Descripción: {service['description']}")
    except:
        print(f"❌ {service['name']}: NO DISPONIBLE")
        print(f"   URL: {service['url']}")
    print()

# Verificar estado del análisis
try:
    analysis_status = requests.get("http://localhost:8000/api/v1/analysis/status", timeout=5).json()
    print("ESTADO DEL ANÁLISIS:")
    print("-" * 40)
    print(f"Estado: {analysis_status['status']}")
    print(f"Análisis disponible: {'Sí' if analysis_status['analysis_available'] else 'No'}")
    print(f"Dependencias OK: {'Sí' if analysis_status['dependencies_ok'] else 'No'}")
    print(f"Análisis activos: {analysis_status['active_analyses']}")
    print(f"Mensaje: {analysis_status['message']}")
except Exception as e:
    print(f"❌ Error obteniendo estado del análisis: {e}")

print("\nACCESO RÁPIDO:")
print("-" * 40)
print("🌐 Aplicación Web: http://localhost:8888")
print("📊 Dashboard: http://localhost:8888/src/app/dashboard/page.tsx")
print("🔧 API Docs: http://localhost:8000/docs")
print("💡 API Health: http://localhost:8000/health")

print("\nCÓMO USAR EL SISTEMA:")
print("-" * 40)
print("1. Abre http://localhost:8888 en tu navegador")
print("2. Ve al Dashboard para subir documentos")
print("3. El sistema analizará automáticamente:")
print("   • Extracción de texto y metadatos")
print("   • Clasificación de secciones")
print("   • Análisis de riesgos")
print("   • Generación de reportes")
print("   • Comparación de propuestas")

print("\nPARA PARAR LOS SERVICIOS:")
print("-" * 40)
print("• Presiona Ctrl+C en las terminales donde están corriendo")
print("• O cierra las terminales directamente")

print(f"\n✅ Sistema verificado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("🎉 ¡Tu sistema de análisis de licitaciones está listo para usar!")

ESTADO FINAL DEL SISTEMA TENDERING AI

SERVICIOS DISPONIBLES:
----------------------------------------
✅ API Backend: ACTIVO
   URL: http://localhost:8000
   Descripción: Análisis de documentos, IA, embeddings

✅ Frontend Web: ACTIVO
   URL: http://localhost:8888
   Descripción: Interfaz de usuario, dashboard

✅ API Documentation: ACTIVO
   URL: http://localhost:8000/docs
   Descripción: Documentación interactiva Swagger

ESTADO DEL ANÁLISIS:
----------------------------------------
Estado: limited
Análisis disponible: No
Dependencias OK: No
Análisis activos: 0
Mensaje: Análisis limitado - verifica dependencias

ACCESO RÁPIDO:
----------------------------------------
🌐 Aplicación Web: http://localhost:8888
📊 Dashboard: http://localhost:8888/src/app/dashboard/page.tsx
🔧 API Docs: http://localhost:8000/docs
💡 API Health: http://localhost:8000/health

CÓMO USAR EL SISTEMA:
----------------------------------------
1. Abre http://localhost:8888 en tu navegador
2. Ve al Dashboard para subir 

# Tendering Analysis System - Lab Notebook

Este notebook demuestra la funcionalidad de los diferentes agentes.



## Classification

In [1]:
import sys
from pathlib import Path
import json
import logging

from utils.agents.document_classification import DocumentClassificationAgent

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    stream=sys.stdout,
                    force=True)

logger = logging.getLogger(__name__)

"""Test básico de clasificación de documentos"""
print("=== Test Básico de Clasificación ===")

# Usar el documento de ejemplo
backend_dir = Path.cwd() 
document_path = backend_dir / ".." / "documents" / "EJEMPLO DE CONTRATO - RETO 1.pdf"
print(f"📄 Ruta del documento de prueba: {document_path}")

# Crear agente de clasificación
db_dir = backend_dir / "db"
agent = DocumentClassificationAgent(
    document_path=document_path,
    vector_db_path=db_dir / "test_classification"
)

try:
    # Procesar documento completo
    report = agent.process_document(provider="auto", force_rebuild=True)
    print(json.dumps(report, indent=2))
    
    if "error" in report:
        print(f"❌ Error en procesamiento: {report['error']}")
    
    # Mostrar resultados
    print("✅ Documento procesado exitosamente")
    print(f"📄 Total de secciones: {report['document_info']['total_sections']}")
    print(f"📝 Total de fragmentos: {report['document_info']['total_fragments']}")
    
    # Mostrar secciones encontradas
    print("\n📋 Secciones clasificadas:")
    for section_name, section_info in report['sections'].items():
        confidence = report['confidence_scores'].get(section_name, 0)
        print(f"  • {section_name}: {section_info['document_count']} fragmentos (Confianza: {confidence:.1f}%)")
    
    # Mostrar requisitos clave encontrados
    if report['key_requirements']:
        print("\n🔍 Requisitos clave encontrados:")
        for section, requirements in report['key_requirements'].items():
            if requirements:
                print(f"  {section}:")
                for req in requirements[:3]:  # Mostrar solo los primeros 3
                    print(f"    - {req[:80]}...")
    
    print("✅ Success")
    
except Exception as e:
    print(f"❌ Error durante el test: {e}")
    import traceback
    traceback.print_exc()

=== Test Básico de Clasificación ===
📄 Ruta del documento de prueba: /home/hackiathon/workspace/tendering_app/backend/../documents/EJEMPLO DE CONTRATO - RETO 1.pdf
2025-08-10 22:34:01,554 - INFO - DSPy DocumentClassificationAgent iniciado con DB: /home/hackiathon/workspace/tendering_app/backend/db/test_classification
2025-08-10 22:34:01,556 - INFO - Iniciando procesamiento completo DSPy del documento: /home/hackiathon/workspace/tendering_app/backend/../documents/EJEMPLO DE CONTRATO - RETO 1.pdf
2025-08-10 22:34:01,561 - INFO - OLLAMA disponible - Versión: 0.11.4
2025-08-10 22:34:01,563 - INFO - OLLAMA disponible - Versión: 0.11.4
2025-08-10 22:34:01,567 - INFO - Modelos OLLAMA disponibles: ['llama3.2:3b', 'llama3.2:1b', 'nomic-embed-text:latest']
2025-08-10 22:34:01,568 - INFO - Usando OLLAMA con modelo: nomic-embed-text
2025-08-10 22:34:01,693 - INFO - Proveedor de embeddings inicializado: ollama (nomic-embed-text)
2025-08-10 22:34:01,697 - INFO - DSPy inicializado con Ollama: llama3.

2025/08/10 22:34:02 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/08/10 22:34:02 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/08/10 22:34:02 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/08/10 22:34:02 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/08/10 22:34:02 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/08/10 22:34:02 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/08/10 22:34:02 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/08/10 22:34:02 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/08/10 22:34

2025-08-10 22:34:03,034 - INFO - Documento clasificado en 2 secciones usando DSPy
2025-08-10 22:34:03,035 - INFO - Procesamiento completo DSPy finalizado exitosamente
{
  "document_info": {
    "source": "/home/hackiathon/workspace/tendering_app/backend/../documents/EJEMPLO DE CONTRATO - RETO 1.pdf",
    "total_sections": 2,
    "total_fragments": 50,
    "classification_timestamp": "2025-08-10 22:34:03.034978",
    "approach": "DSPy + ChromaDB + Standard Chunking",
    "embedding_provider": {
      "provider": "ollama",
      "model": "nomic-embed-text"
    },
    "dspy_enabled": true
  },
  "sections": {
    "OBJETO": {
      "section_name": "OBJETO",
      "document_count": 2,
      "total_characters": 292,
      "content_preview": "OCTAVA \u2013 RESOLUCI\u00d3N DE CONTROVERSIAS\n\u2022\nConciliaci\u00f3n directa\n\u2022\nMediaci\u00f3n ante C\u00e1mara de Comercio de Guayaquil\n\u2022\nJurisdicci\u00f3n ordinaria en Guayaquil...",
      "sources": [
        "EJEMPLO DE CONTRATO - R

Traceback (most recent call last):
  File "<ipython-input-1-7279127bba9c>", line 46, in <cell line: 30>
    confidence = report['confidence_scores'].get(section_name, 0)
KeyError: 'confidence_scores'
